In [1]:
# !pip install scikit-learn-intelex

In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
import joblib
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
# import plotly.offline as pyo

import plotly.io as pio
pio.renderers.default='notebook'

import matplotlib.pyplot as plt
# pyo.init_notebook_mode()

import statsmodels.graphics.tsaplots as tsaplots

import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR

from time import time

In [3]:
X_train = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos/X_train_2y.csv', parse_dates=[0], index_col='datetime_utc')
y_train = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos/y_train_2y.csv', parse_dates=[0], index_col='datetime_utc')
X_test = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos/X_test.csv', parse_dates=[0], index_col='datetime_utc')
y_test = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos/y_test.csv', parse_dates=[0], index_col='datetime_utc')
test_data = pd.read_csv('C:/Users/vmore/notebooks/TFM/Modelos eolica/datos/test_data.csv', parse_dates=[0], index_col='datetime_utc')

# t+1 (2)

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 6)
l_epsilon = [2.**k for k in range(-5, -1)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666,
 0.020833333333333332]

In [7]:
l_epsilon

[0.03125, 0.0625, 0.125, 0.25]

In [8]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t1 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t1.fit(X_train.values, y_train['(t+1)'].values)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                                             StandardScaler()),
                                                                            ('svr',
                                                                             SVR())]),
                                                  transformer=StandardScaler()),
             n_jobs=-1,
             param_grid={'regressor__svr__C': [10.0, 100.0, 1000.0, 10000.0,
                                               100000.0],
                         'regressor__svr__epsilon': [0.03125, 0.0625, 0.125,
                                                     0.25],
                         'regressor__svr__gamma': [0.0026041666666666665,
                                                   0.005208333333333333,
                                                   0.010416666666666666,
                                                   0.020833333333333332]},
             scoring='neg_mean_absolute_error')

In [9]:
best_svr_score_t1 = grid_search_SVM_t1.best_score_
best_svr_model_t1 = grid_search_SVM_t1.best_estimator_

In [10]:
best_svr_model_t1.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=1000.0, epsilon=0.125, gamma=0.0026041666666666665))])

In [11]:
y_pred_svr = best_svr_model_t1.predict(X_test.values)

In [12]:
MAE_SVR_t1=metrics.mean_absolute_error(y_test['(t+1)'], y_pred_svr)
print('MAE en t+1: {:2.2%}'.format(MAE_SVR_t1))

MAE en t+1: 1.04%


In [13]:
joblib.dump(best_svr_model_t1, "modelos/SVR_t1_model.joblib")

['modelos/SVR_t1_model.joblib']

# t+1

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-5, -1)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0013020833333333333,
 0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666]

In [7]:
l_epsilon

[0.03125, 0.0625, 0.125, 0.25]

In [8]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t1 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t1.fit(X_train.values, y_train['(t+1)'].values)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [9]:
best_svr_score_t1 = grid_search_SVM_t1.best_score_
best_svr_model_t1 = grid_search_SVM_t1.best_estimator_

In [10]:
best_svr_model_t1.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=1000.0, epsilon=0.125, gamma=0.0013020833333333333))])

In [11]:
y_pred_svr = best_svr_model_t1.predict(X_test.values)

In [12]:
MAE_SVR_t1=metrics.mean_absolute_error(y_test['(t+1)'], y_pred_svr)
print('MAE en t+1: {:2.2%}'.format(MAE_SVR_t1))

MAE en t+1: 1.32%


In [13]:
joblib.dump(best_svr_model_t1, "modelos/SVR_t1_model.joblib")

['modelos/SVR_t1_model.joblib']

# t+2

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-5, -1)]

In [5]:
l_C

[10.0, 100.0, 1000.0, 10000.0, 100000.0]

In [6]:
l_gamma

[0.0013020833333333333,
 0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666]

In [7]:
l_epsilon

[0.03125, 0.0625, 0.125, 0.25]

In [8]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t2 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t2.fit(X_train.values, y_train['(t+2)'].values)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                                             StandardScaler()),
                                                                            ('svr',
                                                                             SVR())]),
                                                  transformer=StandardScaler()),
             n_jobs=-1,
             param_grid={'regressor__svr__C': [10.0, 100.0, 1000.0, 10000.0,
                                               100000.0],
                         'regressor__svr__epsilon': [0.03125, 0.0625, 0.125,
                                                     0.25],
                         'regressor__svr__gamma': [0.0013020833333333333,
                                                   0.0026041666666666665,
                                                   0.005208333333333333,
                                                   0.010416666666666666]},
             scoring='neg_mean_absolute_error')

In [9]:
best_svr_score_t2 = grid_search_SVM_t2.best_score_
best_svr_model_t2 = grid_search_SVM_t2.best_estimator_

In [10]:
best_svr_model_t2.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100.0, epsilon=0.125, gamma=0.010416666666666666))])

In [11]:
y_pred_svr = best_svr_model_t2.predict(X_test.values)

In [12]:
MAE_SVR_t2=metrics.mean_absolute_error(y_test['(t+2)'], y_pred_svr)
print('MAE en t+2: {:2.2%}'.format(MAE_SVR_t2))

MAE en t+2: 1.69%


In [13]:
joblib.dump(best_svr_model_t2, "modelos/SVR_t2_model.joblib")

['modelos/SVR_t2_model.joblib']

# t+3

In [ ]:
l_C = [10.**k for k in range(1, 4)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, -2)]

In [ ]:
l_gamma

[0.0013020833333333333,
 0.0026041666666666665,
 0.005208333333333333,
 0.010416666666666666]

In [ ]:
l_C

[10.0, 100.0, 1000.0]

In [ ]:
l_epsilon

[0.015625, 0.03125, 0.0625, 0.125]

In [ ]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t3 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t3.fit(X_train.values, y_train['(t+3)'].values)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                                             StandardScaler()),
                                                                            ('svr',
                                                                             SVR())]),
                                                  transformer=StandardScaler()),
             n_jobs=-1,
             param_grid={'regressor__svr__C': [10.0, 100.0, 1000.0],
                         'regressor__svr__epsilon': [0.015625, 0.03125, 0.0625,
                                                     0.125],
                         'regressor__svr__gamma': [0.0013020833333333333,
                                                   0.0026041666666666665,
                                                   0.005208333333333333,
                                                   0.010416666666666666]},
             scoring='neg_mean_absolute_error')

In [ ]:
best_svr_score = grid_search_SVM_t3.best_score_
best_svr_model = grid_search_SVM_t3.best_estimator_

In [ ]:
best_svr_model.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=1000.0, epsilon=0.0625, gamma=0.0026041666666666665))])

In [ ]:
y_pred_svr = best_svr_model.predict(X_test.values)

In [ ]:
MAE_SVR_t3=metrics.mean_absolute_error(y_test['(t+3)'], y_pred_svr)
print('MAE en t+3: {:2.2%}'.format(MAE_SVR_t3))

MAE en t+3: 2.51%


In [ ]:
import joblib

In [ ]:
joblib.dump(best_svr_model, "SVR_t3_model.joblib")

['SVR_t3_model.joblib']

# t+4

In [ ]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, -2)]

In [ ]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t4 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t4.fit(X_train.values, y_train['(t+4)'].values)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                                             StandardScaler()),
                                                                            ('svr',
                                                                             SVR())]),
                                                  transformer=StandardScaler()),
             n_jobs=-1,
             param_grid={'regressor__svr__C': [10.0, 100.0, 1000.0, 10000.0,
                                               100000.0],
                         'regressor__svr__epsilon': [0.015625, 0.03125, 0.0625,
                                                     0.125],
                         'regressor__svr__gamma': [0.0013020833333333333,
                                                   0.0026041666666666665,
                                                   0.005208333333333333,
                                                   0.010416666666666666]},
             scoring='neg_mean_absolute_error')

In [ ]:
best_svr_score_t4 = grid_search_SVM_t4.best_score_
best_svr_model_t4 = grid_search_SVM_t4.best_estimator_

In [ ]:
best_svr_model_t4.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100000.0, epsilon=0.125, gamma=0.0013020833333333333))])

In [ ]:
y_pred_svr = best_svr_model_t4.predict(X_test.values)

In [ ]:
MAE_SVR_t4=metrics.mean_absolute_error(y_test['(t+4)'], y_pred_svr)
print('MAE en t+4: {:2.2%}'.format(MAE_SVR_t4))

MAE en t+4: 3.28%


In [ ]:
joblib.dump(best_svr_model_t4, "SVR_t4_model.joblib")

['SVR_t4_model.joblib']

# t+5

In [ ]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, -2)]

In [ ]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t5 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t5.fit(X_train.values, y_train['(t+5)'].values)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
             estimator=TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                                             StandardScaler()),
                                                                            ('svr',
                                                                             SVR())]),
                                                  transformer=StandardScaler()),
             n_jobs=-1,
             param_grid={'regressor__svr__C': [10.0, 100.0, 1000.0, 10000.0,
                                               100000.0],
                         'regressor__svr__epsilon': [0.015625, 0.03125, 0.0625,
                                                     0.125],
                         'regressor__svr__gamma': [0.0013020833333333333,
                                                   0.0026041666666666665,
                                                   0.005208333333333333,
                                                   0.010416666666666666]},
             scoring='neg_mean_absolute_error')

In [ ]:
best_svr_score_t5 = grid_search_SVM_t5.best_score_
best_svr_model_t5 = grid_search_SVM_t5.best_estimator_

In [ ]:
best_svr_model_t5.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100000.0, epsilon=0.0625, gamma=0.0013020833333333333))])

In [ ]:
y_pred_svr = best_svr_model_t5.predict(X_test.values)

In [ ]:
MAE_SVR_t5=metrics.mean_absolute_error(y_test['(t+5)'], y_pred_svr)
print('MAE en t+5: {:2.2%}'.format(MAE_SVR_t5))

MAE en t+5: 3.98%


In [ ]:
joblib.dump(best_svr_model_t5, "SVR_t5_model.joblib")

['SVR_t5_model.joblib']

In [ ]:
files.download("SVR_t5_model.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# t+6

In [4]:
l_C = [10.**k for k in range(1, 6)]
l_gamma = list( np.array([2.**k for k in range(-6, -2)]) / 12)
l_epsilon = [2.**k for k in range(-6, -2)]

In [5]:
svr = SVR(kernel='rbf')

# Create a pipeline with StandardScaler and SVM
SVM_pipeline = Pipeline([('scaler', StandardScaler()),
                         ('svr', svr)])

# TimeSeries Cross validation
tscv = TimeSeriesSplit(n_splits=2)

y_transformer = StandardScaler()
inner_estimator_svm = TransformedTargetRegressor(regressor=SVM_pipeline,
                                                 transformer=y_transformer)

# Define the parameter grid to search over
param_grid_SVM = {'regressor__svr__C': l_C,
#                   'regressor__svr__C': [0.1, 1, 10, 100],
#                   'regressor__svm__kernel': ['linear', 'rbf', 'poly'],
#                   'regressor__svm__degree': [2, 3, 4],
#                   'regressor__svm__gamma': ['scale', 'auto'],
                  'regressor__svr__gamma': l_gamma,
                  'regressor__svr__epsilon': l_epsilon
                 }

# Create the GridSearchCV object
grid_search_SVM_t6 = GridSearchCV(inner_estimator_svm,
                               param_grid_SVM,
                               refit=True,
                               scoring='neg_mean_absolute_error',
                               cv=tscv,
                               n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search_SVM_t6.fit(X_train.values, y_train['(t+6)'].values)

exception calling callback for <Future at 0x1ac7664ff70 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\parallel.py", line 360, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\parallel.py", line 797, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\parallel.py", line 864, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\parallel.py", line 782, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\vmore\anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
best_svr_score_t6 = grid_search_SVM_t6.best_score_
best_svr_model_t6 = grid_search_SVM_t6.best_estimator_

In [ ]:
best_svr_model_t6.regressor_

Pipeline(steps=[('scaler', StandardScaler()),
                ('svr',
                 SVR(C=100000.0, epsilon=0.125, gamma=0.0013020833333333333))])

In [ ]:
y_pred_svr = best_svr_model_t6.predict(X_test.values)

In [ ]:
MAE_SVR_t6=metrics.mean_absolute_error(y_test['(t+6)'], y_pred_svr)
print('MAE en t+6: {:2.2%}'.format(MAE_SVR_t6))

MAE en t+6: 4.60%


In [ ]:
joblib.dump(best_svr_model_t6, "SVR_t6_model.joblib")

['SVR_t6_model.joblib']

In [ ]:
files.download("SVR_t6_model.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>